# Import an Xspec save file and recover the model parameters

We need to load the save file with pyxspec, then retrieve information from the xspec.AllData and xspec.AllModels objects.

The model parameters of the background components must be interpreted in tandem with their addmodel_* functions for consistency. Because we are building spectral and imaging products for these background components only, we will not consider any additional model components added by the user, who may need those to account for excess emission.

The user is supposed to not modify the loaded data files and existing model components (except their parameter values). For the spectral data files especially, their order must be preserved because this is used to match with their respective region files from bgdinfo.json. The loaded save file should then retain that same order for xspec.AllData.

In [2]:
import xspec

In [3]:
import json
import os
import astropy.io.fits as pf
import nuskybgd
import nuskybgd.model as numodel
from nuskybgd import util

## Load Xspec save file and bgdinfo.json

In [4]:
savefile = 'mymodel_fit_freefcxb.xcm'
nustardir = '/apool/qw/astro/nustar'
specdir = nustardir + '/A1656/70001000002/event_cl/spec'
bgddir = nustardir + '/A1656/70001000002/event_cl/bgd'

In [5]:
os.chdir(bgddir)
xspec.Xset.restore(savefile)


***Warning: The file sent to Xset.restore(): mymodel_fit_freefcxb.xcm
    is not detected to be a file generated from Xset.save().
    Xset.restore() usage is only intended for Xset.save() output.
    General XSPEC/Tcl scripts may not fully execute in PyXspec.




In [6]:
for i in range(xspec.AllData.nSpectra):
    spec = xspec.AllData(i + 1)
    print(spec.fileName)

bgd1A_sr_g30.pha
bgd1B_sr_g30.pha
bgd2A_sr_g30.pha
bgd2B_sr_g30.pha
bgd3A_sr_g30.pha
bgd3B_sr_g30.pha
bgd4A_sr_g30.pha
bgd4B_sr_g30.pha


In [7]:
xspec.AllModels.sources

{2: 'apbgd', 3: 'intbgd', 4: 'fcxb', 5: 'intn', 6: 'icm'}

Now read in the bgdinfo.json file for reference image and region files.

In [8]:
bgdinfo = numodel.check_bgdinfofile('bgdinfo.json')

if bgdinfo is not False:
    # bgfiles and regfiles must have the same ordering
    bgfiles = bgdinfo['bgfiles']
    regfiles = bgdinfo['regfiles']
    refimgf = bgdinfo['refimgf']
    bgdapfiles = bgdinfo['bgdapfiles']
    bgddetfiles = bgdinfo['bgddetfiles']

    bgdapim = {}
    bgdapim['A'] = pf.open(bgdapfiles['A'])[0].data
    bgdapim['B'] = pf.open(bgdapfiles['B'])[0].data

    bgddetim = {}
    bgddetim['A'] = [
        pf.open(bgddetfiles['A'][0])[0].data,
        pf.open(bgddetfiles['A'][1])[0].data,
        pf.open(bgddetfiles['A'][2])[0].data,
        pf.open(bgddetfiles['A'][3])[0].data
    ]
    bgddetim['B'] = [
        pf.open(bgddetfiles['B'][0])[0].data,
        pf.open(bgddetfiles['B'][1])[0].data,
        pf.open(bgddetfiles['B'][2])[0].data,
        pf.open(bgddetfiles['B'][3])[0].data
    ]

In [9]:
bgdinfo

{'bgdapfiles': {'A': 'bgdapA.fits', 'B': 'bgdapB.fits'},
 'bgddetfiles': {'A': ['det0Aim.fits',
   'det1Aim.fits',
   'det2Aim.fits',
   'det3Aim.fits'],
  'B': ['det0Bim.fits', 'det1Bim.fits', 'det2Bim.fits', 'det3Bim.fits']},
 'bgfiles': ['bgd1A_sr_g30.pha',
  'bgd1B_sr_g30.pha',
  'bgd2A_sr_g30.pha',
  'bgd2B_sr_g30.pha',
  'bgd3A_sr_g30.pha',
  'bgd3B_sr_g30.pha',
  'bgd4A_sr_g30.pha',
  'bgd4B_sr_g30.pha'],
 'refimgf': 'bgdapA.fits',
 'regfiles': ['bgd1.reg',
  'bgd1.reg',
  'bgd2.reg',
  'bgd2.reg',
  'bgd3.reg',
  'bgd3.reg',
  'bgd4.reg',
  'bgd4.reg']}

We can check the ordering of the spectra files to make sure they match up.

In [10]:
for i in range(xspec.AllData.nSpectra):
    specfile = xspec.AllData(i + 1).fileName
    if specfile != bgdinfo['bgfiles'][i]:
        print('%s\tError: non-matching %s' % (specfile, bgdinfo['bgfiles'][i]))
    else:
        print('%s\t%s\tOK' % (specfile, bgdinfo['regfiles'][i]))

bgd1A_sr_g30.pha	bgd1.reg	OK
bgd1B_sr_g30.pha	bgd1.reg	OK
bgd2A_sr_g30.pha	bgd2.reg	OK
bgd2B_sr_g30.pha	bgd2.reg	OK
bgd3A_sr_g30.pha	bgd3.reg	OK
bgd3B_sr_g30.pha	bgd3.reg	OK
bgd4A_sr_g30.pha	bgd4.reg	OK
bgd4B_sr_g30.pha	bgd4.reg	OK


## Control parameter links

In [15]:
# Model source names
xspec.AllModels.sources

{2: 'apbgd', 3: 'intbgd', 4: 'fcxb', 5: 'intn', 6: 'icm'}

In [16]:
# Number of spectral files
xspec.AllData.nSpectra

8

In [24]:
# Selecting model source of data group number
m = xspec.AllModels(4, 'apbgd')
# List source components
m.componentNames

['cutoffpl']

In [23]:
# Number of parameters of model source
m.nParameters

3

In [20]:
dir(m)

['_AttrRestrictor__tmpRemoveRestrict',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_getFlux',
 '_getLumin',
 '_getStartParIndex',
 '_handle',
 '_resetRestrict',
 '_turnRestrictOff',
 'componentNames',
 'cutoffpl',
 'energies',
 'expression',
 'flux',
 'folded',
 'lumin',
 'nParameters',
 'name',
 'setPars',
 'show',
 'showList',
 'startParIndex',
 'untie',
 'values']

In [37]:
m.cutoffpl.norm.values

[0.0192684201776, 0.000192684201776, 0.0, 0.0, 1e+20, 1e+24]

In [38]:
m.cutoffpl.norm.link

'= 1.070444e00*apbgd:p6'

In [40]:
m.cutoffpl.norm.untie()

In [41]:
m.show()

In [42]:
xspec.AllModels(2, 'apbgd').show()

In [43]:
m.nParameters

3

## Append the source region to spectra list

This process is similar to how background spectra were added to the list. We append the source this way to enable simultaneous fitting of the background model and the source spectrum. ***Background model is frozen in the output. Select parameters of the background model can be manually thawed as desired.***

In [18]:
def addspec_src(specfiles):
    """
    Append new data files to existing ones, each in their own data
    group. Afterward check the number of loaded spectra against length of
    input list. If not all spectra loaded, an Exception is raised.

    Input:

    specfiles -- List of files to load.
    """
    count_before = xspec.AllData.nSpectra  # Clear any existing loaded data

    # Load each spectrum as a new data group
    for i in range(len(specfiles)):
        xspec.AllData('{num}:{num} {file}'.format(
            num=i + 1 + count_before,  # Xspec spectrum numbering starts at 1
            file=specfiles[i]))

    if xspec.AllData.nSpectra != count_before + len(specfiles):
        raise Exception('Not all requested spectra loaded, cannot proceed!')

In [ ]:
def applymodel_apbgd(presets, refspec, bgdapimwt, model_num, src_number, src_apimwt,
                     model_name='apbgd'):
    """
    Xspec model component 2: apbgd (aperture image background)

    cutoffpl

    p1 (PhoIndex) and p2 (HighCut) are frozen.

    p3 (norm) is defined as

    0.002353
    --------  x  bgdapimwt
       32

    bgdapimwt = np.sum(bgdapim * regmask), for each FPM and region.

    This is calculated for the reference spectra for each FPM. The other
    spectra are scaled using the second part, sum(bgdapim * regmask).

    Inputs:

    presets - Preset model parameter values read from file.

    refspec - Dictionary with 'A' and 'B' keys whose values are the 0-based
        index of the reference spectrum for FPMA and FPMB.

    bgdapimwt - List of sum of aperture image inside background region.

    model_num - Model component number in Xspec, cannot be the same as another
        model or it will replace.
    
    src_number - Position of first source spectrum (index starts at 1)
    
    src_apimwt - List of aperture image weights, must have same number of values
        as the number of spectra beginning at position src_number.

    model_name - Model component name in Xspec, default 'apbgd', cannot be the
        same as another model.
    """
    mod_apbgd = presets['models'][0]['components']
    
    # Input:
#     src_number = 5   # Position of first spectrum
#     src_apimwt = [0.56]
    src_count = len(src_apimwt)
    
    # Validate the spectra count
    if not (xspec.AllData.nSpectra == src_number - 1 + src_count):
        raise Exception('Cannot apply model for spectrum: spectrum number out of range.')

    # Add the response for this source, starting from spectrum number src_number.
    for i in range(src_count):
        s = xspec.AllData(i + src_number)
        s.multiresponse[model_num - 1] = s.response.rmf
        s.multiresponse[model_num - 1].arf = '%s/be.arf' % conf._AUX_DIR

    #xspec.Model('cutoffpl', model_name, model_num)
    # Model already exists when source spectrum is added. Proceed to adjust the params.

    for i in range(src_count):
        spec = xspec.AllData(i + src_number)
        fpm = util.fpm_parse(spec.fileinfo('INSTRUME'))
        m = xspec.AllModels(i + src_number, model_name)
        if False:
#         if i == refspec['A'] or i == refspec['B']:
#             m.cutoffpl.PhoIndex.values = mod_apbgd['cutoffpl'][fpm]['phoindex']
#             m.cutoffpl.HighECut.values = mod_apbgd['cutoffpl'][fpm]['highecut']
#             m.cutoffpl.PhoIndex.frozen = True
#             m.cutoffpl.HighECut.frozen = True
#             m.cutoffpl.norm.values = 0.002353 / 32 * bgdapimwt[i]
        else:
            m.cutoffpl.PhoIndex.link = '%s:p%d' % (
                model_name, 3 * refspec[fpm] + 1)
            m.cutoffpl.HighECut.link = '%s:p%d' % (
                model_name, 3 * refspec[fpm] + 2)
            m.cutoffpl.norm.link = '%e * %s:p%d' % (
                src_apimwt[i] / bgdapimwt[refspec[fpm]],
                model_name,
                3 * refspec[fpm] + 3)

In [ ]:
def applymodel_intbgd(presets, refspec, bgddetimsum, model_num, src_number, src_detimsum,
                    model_name='intbgd'):
    """
    Xspec model component 3: intbgd (instrument background)

    An apec, then many lorentz lines. Each lorentz has 3 parameters (LineE,
    Width, norm); apec has 4 parameters (kT, Abundanc, Redshift, norm).

    For the reference spectra:

    apec params are loaded from preset.

    Lines 1-3 (lorentz, lorentz_3, lorentz_4) are loaded from preset.

    Line 4 (19 keV, lorentz_5) is loaded from preset:

    norm = sum (ifactor * bgddetimsum)
           ---------------------------
                sum (bgddetimsum)


    The other lines are scaled to this one using:

    sum(ifactor * bgddetimsum)

    Other spectra:

    Lines 1-3 scale to refspec lines 1-3 (lorentz, lorentz_3, lorentz_4).

    Line 4 scale to refspec line 4 (lorentz_5).

    Other lines scale to line 4 (lorentz_5).

    apec scale to refspec apec norm.

    Note: due to perculiar model component numbering by XSPEC,
    as in the following,

    Model intbgd:apec<1> + lorentz<2> + lorentz<3> + lorentz<4> + ...

    In this case, the lorentz components have references lorentz, lorentz_3,
    lorentz_4, ... etc. (skipping over lorentz_2). This numbering appears to
    be for all components, not just repeated ones, except the first occurrence
    omits the label.

    Inputs:

    presets - Preset model parameter values read from file.

    refspec - Dictionary with 'A' and 'B' keys whose values are the 0-based
    index of the reference spectrum for FPMA and FPMB.

    bgddetimsum - List of area inside background region for each CCD.

    model_num - Model component number in Xspec, cannot be the same as another
    model or it will replace.

    model_name - Model component name in Xspec, default 'intbgd', cannot be
    the same as another model.
    """
    mod_intbgd = presets['models'][1]['components']

        # Input:
#     src_number = 5   # Position of first spectrum
#     src_apimwt = [0.56]
    src_count = len(src_apimwt)
    
    # Validate the spectra count
    if not (xspec.AllData.nSpectra == src_number - 1 + src_count):
        raise Exception('Cannot apply model for spectrum: spectrum number out of range.')

    # Add the response for this source.
    for i in range(src_count):
        s = xspec.AllData(i + src_number)
        s.multiresponse[model_num - 1] = s.response.rmf

#     xspec.Model('apec' + '+lorentz' * (len(mod_intbgd) - 1),
#                 model_name, model_num)
    # Model already exists when source spectrum is added. Proceed to adjust the params.

    # There are in total (len(mod_intbgd) * 3 + 1) parameters per spectrum. 4
    # parameters for apec and 3 parameters for lorentz.
    for i in range(src_count):
        spec = xspec.AllData(i + src_number)
        fpm = util.fpm_parse(spec.fileinfo('INSTRUME'))
        m = xspec.AllModels(i + src_number, model_name)

        # Reference spectrum
        if False:
#         if i == refspec['A'] or i == refspec['B']:

#             m.apec.kT.values = mod_intbgd['apec'][fpm]['kt']
#             m.apec.Abundanc.values = mod_intbgd['apec'][fpm]['abundanc']
#             m.apec.Redshift.values = mod_intbgd['apec'][fpm]['redshift']
#             m.apec.kT.frozen = True
#             m.apec.Abundanc.frozen = True
#             m.apec.Redshift.frozen = True
#             m.apec.norm.values = np.sum(
#                 bgddetimsum[i] * np.array(
#                     mod_intbgd['apec'][fpm]['ifactors'])
#             ) / np.sum(bgddetimsum[i])

#             # 3 solar lines and 19 keV line -- load them from preset
#             for attr in ['lorentz', 'lorentz_3', 'lorentz_4', 'lorentz_5']:
#                 m_line = getattr(m, attr)
#                 m_line.LineE.values = mod_intbgd[attr][fpm]['linee']
#                 m_line.Width.values = mod_intbgd[attr][fpm]['width']
#                 m_line.LineE.frozen = True
#                 m_line.Width.frozen = True
#                 m_line.norm.values = np.sum(
#                     bgddetimsum[i] * np.array(
#                         mod_intbgd[attr][fpm]['ifactors'])
#                 ) / np.sum(bgddetimsum[i])

#             # All the other lines -- lorentz_6 through lorentz_(components) --
#             # load them from preset and link to 19 keV line (lorentz_5). There
#             # are 4+3*3=13 parameters before lorentz_5.
#             for attr_n in range(5, len(mod_intbgd)):
#                 attr = 'lorentz_%d' % (attr_n + 1)
#                 m_line = getattr(m, attr)
#                 m_line.LineE.values = mod_intbgd[attr][fpm]['linee']
#                 m_line.Width.values = mod_intbgd[attr][fpm]['width']
#                 m_line.LineE.frozen = True
#                 m_line.Width.frozen = True

#                 norm_preset = np.sum(
#                     bgddetimsum[i] * np.array(
#                         mod_intbgd[attr][fpm]['ifactors'])
#                 ) / np.sum(bgddetimsum[i])

#                 lorentz_5_norm_npar = i * m.nParameters + 16

#                 m_line.norm.link = '%e * %s:p%d' % (
#                     norm_preset / m.lorentz_5.norm.values[0],
#                     model_name,
#                     lorentz_5_norm_npar)

        else:
            # Link to ref spectrum
            m_ref = xspec.AllModels(refspec[fpm] + 1, model_name)
            m_ref_npar_offset = m.nParameters * refspec[fpm]

            # Link apec to refspec
            m.apec.kT.link = '%s:p%d' % (
                model_name, m_ref_npar_offset + 1)
            m.apec.Abundanc.link = '%s:p%d' % (
                model_name, m_ref_npar_offset + 2)
            m.apec.Redshift.link = '%s:p%d' % (
                model_name, m_ref_npar_offset + 3)

            norm_preset = np.sum(
                src_detimsum[i] * np.array(
                    mod_intbgd['apec'][fpm]['ifactors'])
            ) / np.sum(src_detimsum[i])
            
            ##################
            # Only in applymodel
            # The reference preset has changed after fitting, so we must
            # calculate its original value for scaling.
            ref_preset = np.sum(
                bgddetimsum[refspec[fpm]] * np.array(
                    mod_intbgd['apec'][fpm]['ifactors'])
            ) / np.sum(bgddetimsum[refspec[fpm]])
            ##################

            m.apec.norm.link = '%e * %s:p%d' % (
                norm_preset / ref_preset,
                model_name,
                m_ref_npar_offset + 4
            )

            # 3 solar lines and 19 keV line -- link to lines in refspec
            for attr in ['lorentz', 'lorentz_3', 'lorentz_4', 'lorentz_5']:
                m_line = getattr(m, attr)
                m_line.LineE.values = mod_intbgd[attr][fpm]['linee']
                m_line.Width.values = mod_intbgd[attr][fpm]['width']
                m_line.LineE.frozen = True
                m_line.Width.frozen = True
                m_line.norm.values = np.sum(
                    bgddetimsum[i] * np.array(
                        mod_intbgd[attr][fpm]['ifactors'])
                ) / np.sum(bgddetimsum[i])

            # All lines --- load values from preset and link to refspec
            for attr_n in range(1, len(mod_intbgd)):
                if attr_n == 1:
                    attr = 'lorentz'  # special case
                else:
                    attr = 'lorentz_%d' % (attr_n + 1)

                m_line = getattr(m, attr)
                m_line.LineE.link = '%s:p%d' % (
                    model_name,
                    m_ref_npar_offset + 3 * (attr_n - 1) + 5)
                m_line.Width.link = '%s:p%d' % (
                    model_name,
                    m_ref_npar_offset + 3 * (attr_n - 1) + 6)

                norm_preset = np.sum(
                    src_detimsum[i] * np.array(
                        mod_intbgd[attr][fpm]['ifactors'])
                ) / np.sum(src_detimsum[i])

                m_line.norm.link = '%e * %s:p%d' % (
                    norm_preset / getattr(m_ref, attr).norm.values[0],
                    model_name,
                    m_ref_npar_offset + 3 * (attr_n - 1) + 7
                )



In [ ]:
src_regfiles = []
src_specfiles = []

In [ ]:
#####################################################
# For background weights: same procedure as when creating bgd model
presets = json.loads(open('%s/ratios.json' % auxildir).read())

instrlist = numodel.get_keyword_specfiles(bgdinfo['bgfiles'],
    'INSTRUME', ext='SPECTRUM')

# Compute aperture image and detector mask based weights using each
# background region's mask
regmask = util.mask_from_region(bgdinfo['regfiles'],
                                bgdinfo['refimgf'])
bgdapim, bgddetim = numodel.load_bgdimgs(bgdinfo)

# Number of det pixels in the region mask.
bgddetweights = numodel.calc_det_weights(bgddetim, regmask, instrlist)
bgddetimsum = bgddetweights['sum']

# Sum of the aperture image in the region mask.
bgdapweights = numodel.calc_ap_weights(bgdapim, regmask, instrlist)
bgdapimwt = bgdapweights['sum']

refspec = numodel.get_refspec(instrlist)
#####################################################

src_instrlist = numodel.get_keyword_specfiles(src_specfiles, 'INSTRUME', ext='SPECTRUM')
src_regmask = util.mask_from_region(src_regfiles, bgdinfo['refimgf'])
src_detweights = numodel.calc_det_weights(bgddetim, src_regmask, src_instrlist)
src_detimsum = src_detweights['sum']
src_apweights = numodel.calc_ap_weights(bgdapim, src_regmask, src_instrlist)
src_apimwt = src_apweights['sum']

## Synthesize the model components

### apbgd component (unfocused aperture image background)

Because this background does not go through the optics, it is unfocused so we assume it has the same spectrum everywhere. The normalization rescaling between any two regions is the ratio of the total predicted flux in them, derived from the aspect-projected aperture background image (bgdapA.fits and bgdapB.fits).

Since all background spectra from each module are tied to one of them (as the reference spectrum) using the preset ratio in ratios.json, we can do this rescaling using any one spectrum from each module. For consistency, we will use the same reference spectra designation as when the model was generated.

### fcxb component (unresolved focused CXB background)

This background component is focused so it is expected to vary across the detector on the scale of the PSF, due to variance in the cosmic X-ray background. Each background region will have a different determination of the FCXB flux there.

By default, we will use an area-weighted average of the FCXB flux in all background regions and model it as constant everywhere. This is done by multiplying the combined aspect-projected detector masks by the average flux, and using this image to infer the total flux in the source region.

For imaging analysis, the user can generate a customized image of the FCXB flux and specify it to be used instead. This is also easily done after the fact by generating an image of just the FCXB component from the save file, and replacing this flux with user's custom image.

### intbgd component (instrumental background)

The instrumental background is modelled to be constant over each detector, and tied to one another by the weights in ratios.json.

We multiply each detector mask image by the flux on that detector, and create two aspect-projected detector background images, one for each module.

### intn component (neutron background)